# Asset Selection

In [1]:
import pandas as pd
import numpy as np

import scipy.stats as stats
from sklearn import covariance

import datetime as dt
import yfinance as yf

import matplotlib.pyplot as plt
import seaborn as sns

import pickle as pk
import requests

import random

jtplot.style(figsize=(15, 9))

In [2]:
with open('../../ntbk/sp500tickers.pickle', 'rb') as f:
    sp500 = pk.load(f)

with open('../../../alphavantage_token.TXT', 'rb') as f:
    TOKEN = f.read().decode('utf-8')

In [3]:
sp500[:10]

['MMM', 'ABT', 'ABBV', 'ABMD', 'ACN', 'ATVI', 'ADBE', 'AMD', 'AAP', 'AES']

In [4]:
url = f'https://www.alphavantage.co/query?function=OVERVIEW&symbol=MMM&apikey={TOKEN}'
r = requests.get(url)
data = r.json()

print(data)

{'Symbol': 'MMM', 'AssetType': 'Common Stock', 'Name': '3M Company', 'Description': 'The 3M Company is an American multinational conglomerate corporation operating in the fields of industry, worker safety, US health care, and consumer goods. The company produces over 60,000 products under several brands, including adhesives, abrasives, laminates, passive fire protection, personal protective equipment, window films, paint protection films, dental and orthodontic products, electrical and electronic connecting and insulating materials, medical products, car-care products, electronic circuits, healthcare software and optical films. It is based in Maplewood, a suburb of Saint Paul, Minnesota.', 'CIK': '66740', 'Exchange': 'NYSE', 'Currency': 'USD', 'Country': 'USA', 'Sector': 'LIFE SCIENCES', 'Industry': 'SURGICAL & MEDICAL INSTRUMENTS & APPARATUS', 'Address': '3M CENTER, BLDG. 220-13E-26A, ST PAUL, MN, US', 'FiscalYearEnd': 'December', 'LatestQuarter': '2024-03-31', 'MarketCapitalization':

In [7]:
data.keys()

dict_keys(['Symbol', 'AssetType', 'Name', 'Description', 'CIK', 'Exchange', 'Currency', 'Country', 'Sector', 'Industry', 'Address', 'FiscalYearEnd', 'LatestQuarter', 'MarketCapitalization', 'EBITDA', 'PERatio', 'PEGRatio', 'BookValue', 'DividendPerShare', 'DividendYield', 'EPS', 'RevenuePerShareTTM', 'ProfitMargin', 'OperatingMarginTTM', 'ReturnOnAssetsTTM', 'ReturnOnEquityTTM', 'RevenueTTM', 'GrossProfitTTM', 'DilutedEPSTTM', 'QuarterlyEarningsGrowthYOY', 'QuarterlyRevenueGrowthYOY', 'AnalystTargetPrice', 'AnalystRatingStrongBuy', 'AnalystRatingBuy', 'AnalystRatingHold', 'AnalystRatingSell', 'AnalystRatingStrongSell', 'TrailingPE', 'ForwardPE', 'PriceToSalesRatioTTM', 'PriceToBookRatio', 'EVToRevenue', 'EVToEBITDA', 'Beta', '52WeekHigh', '52WeekLow', '50DayMovingAverage', '200DayMovingAverage', 'SharesOutstanding', 'DividendDate', 'ExDividendDate'])

In [9]:
data = pd.DataFrame(index=sp500, columns=data.keys())
data.head()

,Symbol,AssetType,Name,Description,CIK,Exchange,Currency,Country,Sector,Industry,...,EVToRevenue,EVToEBITDA,Beta,52WeekHigh,52WeekLow,50DayMovingAverage,200DayMovingAverage,SharesOutstanding,DividendDate,ExDividendDate
MMM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ABT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ABBV,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ABMD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ACN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
# for ticker in sp500[:25]:
#     url = f'https://www.alphavantage.co/query?function=OVERVIEW&symbol={ticker}&apikey={TOKEN}'
#     r = requests.get(url)
#     data.loc[ticker] = r.json()

# data.head()

In [22]:
data.head(25)

,Symbol,AssetType,Name,Description,CIK,Exchange,Currency,Country,Sector,Industry,...,EVToRevenue,EVToEBITDA,Beta,52WeekHigh,52WeekLow,50DayMovingAverage,200DayMovingAverage,SharesOutstanding,DividendDate,ExDividendDate
MMM,MMM,Common Stock,3M Company,The 3M Company is an American multinational co...,66740,NYSE,USD,USA,LIFE SCIENCES,SURGICAL & MEDICAL INSTRUMENTS & APPARATUS,...,1.879,-9.08,1.001,105.31,68.63,101.16,97.91,553361000,2024-06-12,2024-05-23
ABT,ABT,Common Stock,Abbott Laboratories,Abbott Laboratories is an American multination...,1800,NYSE,USD,USA,LIFE SCIENCES,PHARMACEUTICAL PREPARATIONS,...,4.928,18.75,0.718,120.4,88.33,104.05,106.51,1739630000,2024-08-15,2024-07-15
ABBV,ABBV,Common Stock,AbbVie Inc,AbbVie is an American publicly traded biopharm...,1551152,NYSE,USD,USA,LIFE SCIENCES,PHARMACEUTICAL PREPARATIONS,...,6.15,19.45,0.623,179.55,132.11,165.75,162.16,1765870000,2024-08-15,2024-07-15
ABMD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ACN,ACN,Common Stock,Accenture plc,Accenture plc is an Irish-domiciled multinatio...,1467373,NYSE,USD,USA,TRADE & SERVICES,"SERVICES-BUSINESS SERVICES, NEC",...,3.041,18.21,1.227,384.33,277.47,300.63,329.16,626384000,2024-08-15,2024-07-11
ATVI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ADBE,ADBE,Common Stock,Adobe Systems Incorporated,Adobe Inc. is an American multinational comput...,796343,NASDAQ,USD,USA,TECHNOLOGY,SERVICES-PREPACKAGED SOFTWARE,...,10.52,29.33,1.297,638.25,433.97,508.13,547.15,443400000,None,2005-03-24
AMD,AMD,Common Stock,Advanced Micro Devices Inc,"Advanced Micro Devices, Inc. (AMD) is an Ameri...",2488,NASDAQ,USD,USA,MANUFACTURING,SEMICONDUCTORS & RELATED DEVICES,...,11.51,62.93,1.681,227.3,93.11,163.79,152.02,1616310000,None,1995-04-27
AAP,AAP,Common Stock,Advance Auto Parts Inc,"Advance Auto Parts, Inc. (Advance) is an Ameri...",1158449,NYSE,USD,USA,TRADE & SERVICES,RETAIL-AUTO & HOME SUPPLY STORES,...,0.682,18.06,1.226,87.92,47.19,65.92,65.31,59622000,2024-07-26,2024-07-12
AES,AES,Common Stock,The AES Corporation,The AES Corporation is a Fortune 500 company t...,874761,NYSE,USD,USA,ENERGY & TRANSPORTATION,COGENERATION SERVICES & SMALL POWER PRODUCERS,...,2.99,14.85,1.079,22.21,11.08,19.49,17.58,710667000,2024-08-15,2024-08-01


In [28]:
delisted = ['ABMD', 'ATVI', 'ALXN', 'AGN', 'ADS']

- ABMD(Abiomed Inc) - On December 22, 2022, Johnson & Johnson completed the acquisition of Abiomed.
- ATVI(Activision Blizzard Inc) -  its share trading come to a halt due to its acquisition by tech behemoth Microsoft Corporation
- ALXN(Alexion Pharmaceutical, Inc) - British pharmaceutical giant AstraZeneca has acquired American biotech firm and rare diseases specialist Alexion for $39 billion.
- AGN(Allergan Plc) - shares ceased trading as the company's merger with AbbVie (NYSE:ABBV) finally went through
- ADS(Alliance Data Systems) - Market capitalization change

In [32]:
data['Beta'] = data['Beta'].astype(float)
data[data['Beta']>1]

,Symbol,AssetType,Name,Description,CIK,Exchange,Currency,Country,Sector,Industry,...,EVToRevenue,EVToEBITDA,Beta,52WeekHigh,52WeekLow,50DayMovingAverage,200DayMovingAverage,SharesOutstanding,DividendDate,ExDividendDate
MMM,MMM,Common Stock,3M Company,The 3M Company is an American multinational co...,66740,NYSE,USD,USA,LIFE SCIENCES,SURGICAL & MEDICAL INSTRUMENTS & APPARATUS,...,1.879,-9.08,1.001,105.31,68.63,101.16,97.91,553361000,2024-06-12,2024-05-23
ACN,ACN,Common Stock,Accenture plc,Accenture plc is an Irish-domiciled multinatio...,1467373,NYSE,USD,USA,TRADE & SERVICES,"SERVICES-BUSINESS SERVICES, NEC",...,3.041,18.21,1.227,384.33,277.47,300.63,329.16,626384000,2024-08-15,2024-07-11
ADBE,ADBE,Common Stock,Adobe Systems Incorporated,Adobe Inc. is an American multinational comput...,796343,NASDAQ,USD,USA,TECHNOLOGY,SERVICES-PREPACKAGED SOFTWARE,...,10.52,29.33,1.297,638.25,433.97,508.13,547.15,443400000,None,2005-03-24
AMD,AMD,Common Stock,Advanced Micro Devices Inc,"Advanced Micro Devices, Inc. (AMD) is an Ameri...",2488,NASDAQ,USD,USA,MANUFACTURING,SEMICONDUCTORS & RELATED DEVICES,...,11.51,62.93,1.681,227.3,93.11,163.79,152.02,1616310000,None,1995-04-27
AAP,AAP,Common Stock,Advance Auto Parts Inc,"Advance Auto Parts, Inc. (Advance) is an Ameri...",1158449,NYSE,USD,USA,TRADE & SERVICES,RETAIL-AUTO & HOME SUPPLY STORES,...,0.682,18.06,1.226,87.92,47.19,65.92,65.31,59622000,2024-07-26,2024-07-12
AES,AES,Common Stock,The AES Corporation,The AES Corporation is a Fortune 500 company t...,874761,NYSE,USD,USA,ENERGY & TRANSPORTATION,COGENERATION SERVICES & SMALL POWER PRODUCERS,...,2.99,14.85,1.079,22.21,11.08,19.49,17.58,710667000,2024-08-15,2024-08-01
A,A,Common Stock,Agilent Technologies Inc,"Agilent Technologies, Inc. is an American anal...",1090872,NYSE,USD,USA,LIFE SCIENCES,INSTRUMENTS FOR MEAS & TESTING OF ELECTRICITY ...,...,6.24,24.88,1.049,155.06,96.3,137.01,132.71,291761000,2024-07-24,2024-07-02
ALK,ALK,Common Stock,Alaska Air Group Inc,Alaska Air Group is an airline holding company...,766421,NYSE,USD,USA,ENERGY & TRANSPORTATION,"AIR TRANSPORTATION, SCHEDULED",...,0.693,8.32,1.604,54.66,30.75,41.18,38.71,126892000,2020-03-05,2020-02-14
ALB,ALB,Common Stock,Albemarle Corp,Albemarle Corporation is a fine chemical manuf...,915913,NYSE,USD,USA,LIFE SCIENCES,"PLASTIC MATERIALS, SYNTH RESINS & NONVULCAN EL...",...,1.851,22.45,1.575,218.5,90.32,112.38,124.93,117527000,2024-07-01,2024-09-13
ARE,ARE,Common Stock,Alexandria Real Estate Equities Inc,"Alexandria Real Estate Equities, Inc. is an Am...",1035443,NYSE,USD,USA,REAL ESTATE & CONSTRUCTION,REAL ESTATE INVESTMENT TRUSTS,...,11.23,22.36,1.162,131.26,87.93,118.84,117.75,174883000,2024-07-15,2024-06-28


In [33]:
data[data['Beta']>1].groupby('Sector').count()

,Symbol,AssetType,Name,Description,CIK,Exchange,Currency,Country,Industry,Address,...,EVToRevenue,EVToEBITDA,Beta,52WeekHigh,52WeekLow,50DayMovingAverage,200DayMovingAverage,SharesOutstanding,DividendDate,ExDividendDate
Sector,,,,,,,,,,,,,,,,,,,,,
ENERGY & TRANSPORTATION,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
LIFE SCIENCES,4,4,4,4,4,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4
MANUFACTURING,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
REAL ESTATE & CONSTRUCTION,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
TECHNOLOGY,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
TRADE & SERVICES,3,3,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3


In [14]:
data.to_csv('../data/sp500_Company_Overview.csv')